In [ ]:
# Quelle für Releases: https://releases.llvm.org/
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import logging
import semver
import datetime
import re
import numpy as np
import matplotlib.dates as mdates

input_file=Path("website_versions.csv")
assert input_file.exists()

In [ ]:
df = pd.read_csv(input_file)
# patch version in semver is only optional beginning with version 3, but there is no official release yet. Therefore missing patch versions will be set to 0
modified_versions = df.Version.apply(lambda x: x if re.match("\d+\.\d+\.\d+",x) else x + ".0")
df.Version = modified_versions.apply(semver.VersionInfo.parse)
df.Date= pd.to_datetime(df.Date)
df.head()

In [ ]:
df.describe()

In [ ]:
# Choose some nice levels
# Source: https://matplotlib.org/3.1.0/gallery/lines_bars_and_markers/timeline.html
levels = np.tile([-5, 5, -3, 3, -1, 1],
                 int(np.ceil(len(df.Version)/6)))[:len(df.Version)]


In [ ]:
levels2=[]
for version in df.Version:
    if version.minor == 0 and version.patch == 0:
        levels2.append(5)
    elif version.patch == 0:
        levels2.append(-5)
    else:
        levels2.append(-3)

In [ ]:
list(df.Version.groupby(by=(lambda x: df.Version.iloc[x].major)))

In [ ]:
df.Version.iloc[0].major

In [ ]:

# Create figure and plot a stem plot with the date
fig, ax = plt.subplots(figsize=(8.8, 4), constrained_layout=True)
ax.set(title="LLVM release dates")

markerline, stemline, baseline = ax.stem(df.Date, levels2,
                                         linefmt="C3-", basefmt="k-",
                                         use_line_collection=True)

plt.setp(markerline, mec="k", mfc="w", zorder=3)

# Shift the markers to the baseline by replacing the y-data by zeros.
#markerline.set_ydata(np.zeros(len(df.Version)))

# annotate lines
# vert = np.array(['top', 'bottom'])[(levels > 0).astype(int)]
# for d, l, r, va in zip(df.Date, levels, df.Version, vert):
#     ax.annotate(r, xy=(d, l), xytext=(-3, np.sign(l)*3),
#                 textcoords="offset points", va=va, ha="right")

# format xaxis with 4 month intervals
# ax.get_xaxis().set_major_locator(mdates.MonthLocator(interval=12))
ax.get_xaxis().set_major_locator(mdates.YearLocator())
ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%b %Y"))
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

# remove y axis and spines
ax.get_yaxis().set_visible(False)
for spine in ["left", "top", "right"]:
    ax.spines[spine].set_visible(False)

ax.margins(y=0.1)

In [ ]:
levels